# W04A — Cómo “piensa” el motor: **EXPLAIN**, cardinalidad y performance (DuckDB)

**Objetivo:** aprender a **leer planes de ejecución** y detectar los 3 problemas más comunes en analítica:
1) **scan enorme** (lees más de lo que necesitas),
2) **JOIN que infló** (cardinalidad mal controlada),
3) **agregación cara** (filtro tarde / columnas de más).

## Bibliografía (W04 en general)

### Libro guía — **DDIA (Martin Kleppmann)**
- **Capítulo 3: Storage and Retrieval**  
  Úsalo para: *OLTP vs OLAP, por qué en analítica importa el ancho de banda, column-oriented storage, y por qué el layout/índices cambian el costo de las consultas.*


### Documentación práctica — DuckDB
- `EXPLAIN` / `EXPLAIN ANALYZE` (planes y cardinalidad)
- Profiling con `PRAGMA enable_profiling`
- Índices (`CREATE INDEX`) y tipos de índices (zonemap / ART)


In [ ]:
from pathlib import Path
import duckdb

PROJECT_ROOT = Path(".").resolve()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
DB_PATH = PROJECT_ROOT / "data" / "exoplanets.duckdb"
ART_DIR = PROJECT_ROOT / "artifacts"
DOCS_DIR = PROJECT_ROOT / "docs"

RAW_DIR.mkdir(parents=True, exist_ok=True)
ART_DIR.mkdir(parents=True, exist_ok=True)
DOCS_DIR.mkdir(parents=True, exist_ok=True)

con = duckdb.connect(str(DB_PATH))

raw_csv = RAW_DIR / "pscomppars.csv"
if not raw_csv.exists():
    raise FileNotFoundError(
        f"No encuentro {raw_csv}. Necesitas el CSV de W01/W02."
    )

def sql_quote(s: str) -> str:
    return "'" + s.replace("'", "''") + "'"

con.execute(f'''
CREATE OR REPLACE VIEW raw_ps AS
SELECT * FROM read_csv_auto({sql_quote(str(raw_csv.resolve()))})
''')

# Si ya tienes Silver/Facts de W03B, perfecto. Si no, los construimos rápido aquí (idéntico a W03B).
con.execute("DROP TABLE IF EXISTS silver_planet")
con.execute('''
CREATE TABLE silver_planet AS
SELECT
  pl_name,
  hostname,
  discoverymethod,
  disc_year,
  sy_snum,
  sy_pnum,
  sy_dist,
  ra,
  dec,
  pl_orbper,
  pl_rade,
  pl_bmasse,
  pl_eqt,
  st_teff,
  st_rad,
  st_mass
FROM raw_ps
WHERE pl_name IS NOT NULL
  AND hostname IS NOT NULL
  AND (disc_year IS NULL OR (disc_year BETWEEN 1980 AND 2026))
  AND (pl_rade  IS NULL OR (pl_rade  > 0 AND pl_rade  <= 30))
  AND (pl_bmasse IS NULL OR (pl_bmasse > 0))
''')

con.execute("DROP TABLE IF EXISTS dim_host_full")
con.execute('''
CREATE TABLE dim_host_full AS
SELECT
  hostname,
  MAX(sy_dist)  AS sy_dist,
  MAX(ra)       AS ra,
  MAX(dec)      AS dec,
  MAX(st_teff)  AS st_teff,
  MAX(st_rad)   AS st_rad,
  MAX(st_mass)  AS st_mass
FROM silver_planet
GROUP BY hostname
''')

con.execute("DROP TABLE IF EXISTS fact_planet")
con.execute('''
CREATE TABLE fact_planet AS
SELECT DISTINCT
  pl_name,
  hostname,
  discoverymethod,
  disc_year,
  pl_orbper,
  pl_rade,
  pl_bmasse,
  pl_eqt
FROM silver_planet
''')

con.sql("SELECT COUNT(*) AS n_fact, COUNT(DISTINCT pl_name) AS n_pl, COUNT(DISTINCT hostname) AS n_hosts FROM fact_planet").show()

## 1) Qué vamos a mirar en un plan (lectura rápida)

Cuando mires `EXPLAIN`, busca:
- **SCAN**: ¿está leyendo demasiadas filas/columnas?
- **FILTER**: ¿se aplica temprano o tarde?
- **JOIN**: ¿qué tipo? (HASH JOIN típico en analítica)
- **GROUP BY**: ¿cuántas filas entran?
- **CARDINALIDAD**: cuántas filas pasan por cada operador

`EXPLAIN` = estimado  
`EXPLAIN ANALYZE` = real (ejecuta y muestra cardinalidad real + tiempos)

In [ ]:
# TU TURNO 1: pega aquí una consulta tipo "métrica" y mira su plan.
# Debe tener: WHERE + GROUP BY (como en W02A).
# Ejemplo de idea: por disc_year o por discoverymethod.

q = '''
-- TODO: tu SQL
SELECT 1;
'''

con.sql("EXPLAIN " + q).show()
# Si corre rápido:
# con.sql("EXPLAIN ANALYZE " + q).show()

In [ ]:
# TU TURNO 2: escribe dos consultas equivalentes:
# A) con SELECT * y B) con solo 3 columnas.
# Luego compara sus planes.

qA = '''
-- TODO: SELECT * ...
SELECT 1;
'''
qB = '''
-- TODO: SELECT col1, col2, col3 ...
SELECT 1;
'''

con.sql("EXPLAIN " + qA).show()
con.sql("EXPLAIN " + qB).show()

In [ ]:
# TU TURNO 3: valida un JOIN sano con evidencia
# 1) arma un JOIN fact_planet + dim_host_full (LEFT JOIN)
# 2) muestra EXPLAIN
# 3) muestra n_fact vs n_join

q_join = '''
-- TODO: tu SQL de join + group by (o join simple)
SELECT 1;
'''

con.sql("EXPLAIN " + q_join).show()

n_fact = con.sql("SELECT COUNT(*) FROM fact_planet").fetchone()[0]
n_join = con.sql('''
-- TODO: cuenta filas del join (LEFT JOIN)
SELECT 1;
''').fetchone()[0]

n_fact, n_join

## Para entregar (W04A) — mesurado

### En clase
1) `docs/w04a_perf_report.md` con:
   - 2 planes (`EXPLAIN`) pegados (consulta tuya + JOIN)
   - 2 conclusiones cortas (2–3 líneas cada una) sobre:
     - dónde está el costo (SCAN/JOIN/GROUP BY)
     - qué mejorarías (leer menos columnas, filtrar antes, etc.)
2) `docs/decisions_log.md`: 1 decisión:
   - “Reescribí una consulta para reducir costo” + evidencia (plan antes/después o conteo).

### Tarea
1) Ejecuta `EXPLAIN ANALYZE` sobre 1 consulta y pega el resultado.
2) Exporta a `artifacts/` un archivo de evidencia:
   - `artifacts/w04a_explain_q1.txt` (copiar/pegar plan a texto está bien)

## Reflexión (bitácora)
- ¿Qué parte del plan te costó más interpretar?
- Si el dataset creciera 100×, ¿qué operador crees que empeora primero (SCAN/JOIN/GROUP BY) y por qué?